In [ ]:
# Elijo como metodo de feature selection el arbol de regresión
# Codifico variables categóricas en números (Region, Producto, Mes)
# Entreno un Arbol de regresión para estimar importancia de estas variables
# Muestro y rankeo las variables de acuerdo a su importancia



from google.colab import files
import pandas as pd


uploaded = files.upload()
ventas_dataset = pd.read_csv("ventas_dataset.csv")



import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in ["Region", "Producto", "Mes"]:
    le = LabelEncoder()
    ventas_dataset[col] = le.fit_transform(ventas_dataset[col])
    label_encoders[col] = le

X = ventas_dataset.drop(columns=["Ventas"])
y = ventas_dataset["Ventas"]

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

importances = model.feature_importances_
feature_names = X.columns

feature_importance_df = pd.DataFrame({"Variable": feature_names, "Importancia": importances})
feature_importance_df = feature_importance_df.sort_values(by="Importancia", ascending=False)

selected_features = feature_importance_df[feature_importance_df["Importancia"] > 0.1]["Variable"]
X_selected = ventas_dataset[selected_features]

print(feature_importance_df)

Saving ventas_dataset.csv to ventas_dataset (3).csv
               Variable  Importancia
4       Costo_Operativo     0.319840
3  Satisfaccion_Cliente     0.319498
2                   Mes     0.189748
0                Region     0.099135
1              Producto     0.071778


In [ ]:
# Entreno un modelo re predicción RFR
# Hago predicciones sobre los datos de prueba
#Imprimo las predicciones para compararlas con datos reales, y reviso el error de la preducción


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from google.colab import files
import pandas as pd

uploaded = files.upload()
ventas_dataset = pd.read_csv("ventas_dataset.csv")

import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print("Primeras predicciones vs. valores reales:")
print("Predicciones: ", y_pred[:10])
print("Valores reales: ", y_test[:10].values)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nMétricas del modelo:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.2f}")

# En conclusión, por las metricas obetindas, vemos que el modelo no esta prediciendo bien. Procedemos a eliminar outliers reemplazando por media.

Saving ventas_dataset.csv to ventas_dataset (7).csv
Primeras predicciones vs. valores reales:
Predicciones:  [2650.13 2125.6  2884.09 3510.22 2220.46 3261.26 3393.19 3416.82 3023.25
 3042.32]
Valores reales:  [3806 3253 3049 4954 2434 2899 1369 2153 2733 4499]

Métricas del modelo:
MAE: 1030.41
MSE: 1454573.20
R²: -0.21


In [ ]:
import numpy as np
import pandas as pd


media_ventas = ventas_dataset['Ventas'].mean()


Q1 = ventas_dataset['Ventas'].quantile(0.25)
Q3 = ventas_dataset['Ventas'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR


ventas_dataset['Ventas'] = ventas_dataset['Ventas'].apply(
    lambda x: media_ventas if (x < limite_inferior or x > limite_superior) else x
)


print(ventas_dataset.head())

  Region    Producto      Mes  Ventas  Satisfaccion_Cliente  Costo_Operativo
0  Norte  Producto_A  2023-01    4174              4.593086             1794
1  Norte  Producto_A  2023-02    2130              4.559382             2138
2  Norte  Producto_A  2023-03    3169              3.311989              830
3  Norte  Producto_A  2023-04    2482              4.732352              630
4  Norte  Producto_A  2023-05    2685              3.112823             2891


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)


y_pred = regressor.predict(X_test)


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print("\nMétricas del modelo después de eliminar outliers:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.2f}")

# Los resultados de las metricas no mejoraron. Pruebo con otro modelo de predicción, KNeighbors


Métricas del modelo después de eliminar outliers:
MAE: 1030.41
MSE: 1454573.20
R²: -0.21


In [ ]:
from sklearn.neighbors import KNeighborsRegressor


knn_regressor = KNeighborsRegressor(n_neighbors=5)
knn_regressor.fit(X_train, y_train)


y_pred_knn = knn_regressor.predict(X_test)


mae_knn = mean_absolute_error(y_test, y_pred_knn)
mse_knn = mean_squared_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)


print("\nMétricas del modelo KNN:")
print(f"MAE: {mae_knn:.2f}")
print(f"MSE: {mse_knn:.2f}")
print(f"R²: {r2_knn:.2f}")


Métricas del modelo KNN:
MAE: 874.46
MSE: 1213607.66
R²: -0.01


In [ ]:
# En conclusión, vemos una mejora considerable en las metricas al usar KNN, esto se debe a que las relaciones entre variables no son lineales.